# Using Symmetry Function Representation of Molecules to predict Atomization Energies of the molecules 

Reference :  https://pubs.rsc.org/en/content/articlelanding/2017/sc/c6sc05720a
Dataset: http://quantum-machine.org/datasets/ (QM 7)

Running the code : use appropriate addresses for the file while loading dataset(qm7.mat) and feature vectors(feature_vector5.mat)
    Comments are provided at the appropriate section of the code

In [1]:
import scipy.io
import numpy as np
import sys
import math
from math import exp
from math import cos
from math import sqrt
from math import *
from sklearn.preprocessing import normalize

In [17]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy
import scipy.io
import numpy as np
from torch.autograd import Variable
import torch.optim as optim
from sklearn.preprocessing import normalize
from sklearn.base import TransformerMixin
from sklearn.preprocessing import StandardScaler
import random

Below cell contains code for encoding the atoms H,C,N,O and S from their atomic numbers
1 - H, 2 - C, 3 - N,4 - O, 5 - S, 0 - No atom 

In [2]:
def switch_demo(argument):
    switcher = {
        0: 0,
        1: 1,
        6: 2,
        7: 3,
        8: 4,
        
    }
    return switcher.get(argument, 5)

# Functions for Symmetry function Calculations

Cutoff for radial symmetry

In [3]:
def cutoffradial(Rij) :
    if (Rij >8.1) :
        return 0.0

    return (0.5*cos((3.14*Rij)/8.1) + 0.5)

Cutoff for angular symmetry

In [ ]:
def cutoffangular(Rij) :
    if (Rij > 8.1) :
        return 0.0

    return (0.5*cos((3.14*Rij)/8.1) + 0.5)


Radial Symmetry Function

In [ ]:
def radial(Ri,Rj,Rs,eta) :
    Rij = math.sqrt((Ri[0]-Rj[0])*(Ri[0]-Rj[0]) + (Ri[1]-Rj[1])*(Ri[1]-Rj[1]) + (Ri[2]-Rj[2])*(Ri[2]-Rj[2]))
    cc = cutoffradial(Rij)
    exx = exp((((-1)*(Rij-3.9)*(Rij-3.9))))
    radf = exp((((-1)*(Rij-3.9)*(Rij-3.9))))*cc

    return (radf)

Dot product

In [4]:
def dot(Vi, Vj) :
    return	abs(Vi[0]*Vj[0] + Vi[1]*Vj[1] + Vi[2]*Vj[2])

Norm

In [5]:
def norm(Vi) :
    return sqrt(dot(Vi, Vi))

Angular Symmetry Function

In [6]:
def angular(Ri,Rj,Rk,Rs,eta,theta,zeta) :

    Ri = np.array(Ri)
    Rj = np.array(Rj)
    Rk = np.array(Rk)
    Vj = Rj - Ri
    Vk = Rk - Ri

    cosan = dot(Vj, Vk) / (norm(Vj) * norm(Vk))
    cosan = round(cosan,5)
    try :
        acos(cosan)

    except :
        print(cosan)
        sys.exit()

    angle = acos(cosan)

    result = pow(2, 1-3)*pow( 1+cos(angle-theta),zeta )
    result = result * exp(-1*eta*pow( (norm(Vj)+norm(Vk))/2 - Rs , 2) )
    result = result * cutoffangular(norm(Vj)) * cutoffangular(norm(Vk))
    result = round(result,5)
    return result

# Dataloader  - dataset QM7

In [7]:
#############$$$$$$$$$$$$$$$$$$$$$$ Below use the address of your qm7.mat downloaded file $$$$$########
qm7 = scipy.io.loadmat('/home/mehthab/Downloads/qm7.mat')

#print(str(qm7))

X = np.array(qm7['X'])
R = np.array(qm7['R'])
T = np.array(qm7['T'])
P = np.array(qm7['P'])
Z = np.array(qm7['Z'])

#X is Coulomb Matriz
#T is Atomization Energies - labels
#P is splits for cross-validation
#Z is atomic charge of each atom in the molecule
#R is cartesian coordinates of each atom in the molecule

xS = np.shape(X)
rS = np.shape(R)
tS = np.shape(T)
zS = np.shape(Z)
pS = np.shape(P)


Parameters for symmetry functions

In [8]:
Rs = [0.50,1.17,1.83,2.50,3.17,3.83,4.50,5.17]
eta = 4.00
zeta = 8.00
the = [0.00,1.57,3.14,4.71]
Ar = np.zeros([23,520,7165])

rc = np.zeros([7165,23])

Start indices for sub-AEvs of AEvs

In [9]:
a1 = [0,8,16,24,32]
a2 = np.zeros([5,5])

a2[0][0] = 40
a2[0][1] = 72
a2[0][2] = 104
a2[0][3] = 136
a2[0][4] = 168
a2[1][1] = 200
a2[1][2] = 232
a2[1][3] = 264
a2[1][4] = 296
a2[2][2] = 328
a2[2][3] = 360
a2[2][4] = 392
a2[3][3] = 424
a2[3][4] = 456
a2[4][4] = 488

a2[0][0] = 40
a2[1][0] = 72
a2[2][0] = 104
a2[3][0] = 136
a2[4][0] = 168
a2[1][1] = 200
a2[2][1] = 232
a2[3][1] = 264
a2[4][1] = 296
a2[2][2] = 328
a2[3][2] = 360
a2[4][2] = 392
a2[3][3] = 424
a2[4][3] = 456
a2[4][4] = 488


# Calculating AEVs

In [ ]:
for i in range(7165) :
    for j in range(23) :
        u = int(Z[i][j])
        v = switch_demo(u)
        rc[i][j] = v
        rc[i][j] = int(rc[i][j])

for k in range(7165) :
    for i in range(23) :
        if(int(rc[k][i])==0) :
            continue

        for j in range(23) :
            if (i==j) :
                continue
            if (int(rc[k][j]) == 0) :
                continue

            start = a1[int(rc[k][j])-1]

            for y in range(8) :
                uu = radial(R[k][i],R[k][j],Rs[y],eta)
                Ar[i][start + y][k] = Ar[i][start + y][k] + uu

            for y2 in  range((j+1),23):
                if (y2==i) :
                    continue
                if (y2==j) :
                    continue
                if (int(rc[k][y2]) == 0) :
                    continue

                start = a2[int(rc[k][j]-1)][int(rc[k][y2]-1)]

                for y3 in range(8) :
                    for y4 in range(4) :
                        uvv = angular(R[k][i],R[k][j],R[k][y2],Rs[y3],eta,the[y4],zeta)

                        Ar[i][int(start + int((y3+1)*(y4+1) - 1))][k] = Ar[i][int(start + int((y3+1)*(y4+1) - 1))][k] + uvv

Normalizing obtained AEVs(feature vectors) and storing to .mat file

In [ ]:
print(np.shape(Ar))

Ar = np.transpose(Ar)

print(np.shape(Ar))
Ar =Ar*1000

for k in range(7165) :
    aa = Ar[k]
    aa = np.transpose(aa)
    aa[0:23] = normalize(aa[0:23])
    aa = np.transpose(aa)
    Ar[k] = aa

Ar = Ar*1000
Ar = np.round(Ar,decimals = 3)

aa = Ar[70]
aa = np.transpose(aa)

Ar = np.transpose(Ar)
print(np.shape(Ar))
print(np.shape(rc))
print(np.shape(T))
scipy.io.savemat('feature_vector5.mat', {'AEVs' : Ar , 'Atomic_Num' : rc , 'labels' : T},do_compression = True)

# Feature Vectors Preprocessing (Normalization)

In [18]:
class NDStandardScaler(TransformerMixin):
    def __init__(self, **kwargs):
        self._scaler = StandardScaler(copy=True, **kwargs)
        self._orig_shape = None

    def fit(self, X, **kwargs):
        X = np.array(X)
        # Save the original shape to reshape the flattened X later
        # back to its original shape
        if len(X.shape) > 1:
            self._orig_shape = X.shape[1:]
        X = self._flatten(X)
        self._scaler.fit(X, **kwargs)
        return self

    def transform(self, X, **kwargs):
        X = np.array(X)
        X = self._flatten(X)
        X = self._scaler.transform(X, **kwargs)
        X = self._reshape(X)
        return X

    def _flatten(self, X):
        # Reshape X to <= 2 dimensions
        if len(X.shape) > 2:
            n_dims = np.prod(self._orig_shape)
            X = X.reshape(-1, n_dims)
        return X

    def _reshape(self, X):
        # Reshape X back to it's original shape
        if len(X.shape) >= 2:
            X = X.reshape(-1, *self._orig_shape)
        return X

Feature Vectors Loading 

In [19]:
#######$$$$$$$$$$$$$$ Below use the address of your feature vector (.mat) file $$$$$$$$###########
# Get input feature dataset
features = scipy.io.loadmat('/home/mehthab/feature_vector5.mat')

rans = np.arange(7165)

# Turn feature dataset into seperate arrays
AEVs = np.transpose(np.array(features['AEVs']), (2, 0, 1))
Atomic_Num = np.array(features['Atomic_Num'], dtype=np.long)
Target = np.array(features['labels'][0])

AEVs = np.round(AEVs,4)
print("Shapes:")
print(np.shape(AEVs))
print(np.shape(Atomic_Num))
print(np.shape(Target))
print(AEVs[50][2][41:80])
list_atoms = []
for row in Atomic_Num:
    for elem in row:
        list_atoms.append(elem)
print(np.unique(np.array(list_atoms)))

Shapes:
(7165, 23, 520)
(7165, 23)
(7165,)
[0.00000e+00 9.55000e-01 6.23990e+01 1.21611e+02 2.07960e+01 1.80670e+01
 7.98560e+01 0.00000e+00 1.57062e+02 0.00000e+00 3.21346e+02 0.00000e+00
 2.44688e+02 5.30000e-02 5.83330e+01 0.00000e+00 4.01000e-01 0.00000e+00
 0.00000e+00 2.31000e-01 0.00000e+00 0.00000e+00 1.20000e-02 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 8.50000e-02 1.75900e+00
 1.35310e+01 7.00950e+01 1.24182e+02]
[0 1 2 3 4 5]


Feature Vector Normalization

In [20]:
Ar= AEVs
scaler = NDStandardScaler()
data = scaler.fit_transform(Ar)
AEVs = data
print(np.shape(data))

(7165, 23, 520)


In [21]:
print(data[50][3][280:320])

[ 0.         -0.3606426   0.         -0.13491457 -0.32410168  0.
  0.         -0.27383865  0.          0.          0.         -0.18444263
  0.          0.          0.         -0.28079271  0.         -0.01670097
 -0.02434471 -0.02938443 -0.11238703 -0.14150766 -0.15390373 -0.10388758
 -0.06607923 -0.12222848  0.         -0.15160531  0.         -0.14719748
 -0.07397864 -0.11991937  0.         -0.115648    0.         -0.01700178
 -0.06798901  0.          0.         -0.0644283 ]


# Neural Net Model

In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(520, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128,64)
        self.fc5 = nn.Linear(64, 1)
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        x = (self.fc5(x))
        return x #need activation function on x or loss function

# Neural Net Initialization - Training - Testing

Initializing 5 instances of class Net - one each for H,C,N,O and S
Training on 5000 molecules
Testing on 2001 molecules

In [23]:
def doNN():
    # Declare Nets
    NNP_H = Net()
    NNP_C = Net()
    NNP_N = Net()
    NNP_O = Net()
    NNP_S = Net()
    
    # put in a list for ease of access
    nets = [NNP_H, NNP_C, NNP_N, NNP_O, NNP_S]
    
    Los = []

    # corresponding optimizers
    optimizers = []
    #criterions = []
    for net in nets:
        optimizers.append( optim.Adam(net.parameters(), lr=0.0099,betas=(0.5,0.59) ,eps=1e-08))
    
    criterion = nn.L1Loss()
    
    ##########################################################################################
    # training
    epochs = 20
    molecules = 5000
    batch_size = 20
    rep = int(molecules/batch_size)
    
    for epoch in range(epochs) :
        print("epoch: ", epoch)
        
        for mols in range(rep) :
            
            outp = torch.zeros(1,batch_size)
            toget = torch.zeros(1,batch_size)
            ck = int(0)
            
            start = mols*batch_size
            
            for mole in range(start,start+batch_size):
                
                molecule = int(rans[int(mole)])
                molecule = mole

                out_f = torch.zeros(1,1)

                # use all relevant aev's on the relevant nets
                for atom in range(23):
                    if(Atomic_Num[molecule][atom]==0):
                        continue

                    aev = torch.from_numpy(AEVs[molecule][atom])
                    out = nets[Atomic_Num[molecule][atom]-1](aev.float())
                    out_f = out_f + out

                outp[0][ck] = out_f
                
                targett = Target[molecule]
                targett = torch.from_numpy(np.array(targett))
                targett = targett.float()
                toget[0][ck] = targett
                ck = int(ck+1)
                
                used_atoms = []
                for atom in range(23):
                    if(Atomic_Num[molecule][atom]!=0):
                        used_atoms.append(Atomic_Num[molecule][atom])
                used_atoms = np.unique(np.array(used_atoms))
            
            # if no nets were used, just move on to the next molecule
                if(len(used_atoms)==0) :
                    print("scream")
                    break
                
            # setting the parameters for the entire net to be zero
            for net in nets:
                net.zero_grad()
            
            for optimizer in optimizers:
                    optimizer.zero_grad()

            toget = toget.view(1,-1)
            # use loss function
            loss = criterion(outp,toget)

            # backpropagate
            loss.backward()
            
            # step the optimizers            
            for atom in range(5) :
                optimizers[atom].step()
    
    
    ###################################################################################
    # Testing
    losses = []
    answ = 0
    for mole in range(5501,7100) :
        molecule = int(rans[int(mole)])
        molecule = mole

        molecule_out = torch.zeros(1,1)
        for atom in range(23):
            if(Atomic_Num[molecule][atom]==0):
                continue
            aev = torch.from_numpy((AEVs[molecule][atom]))
            net_out = nets[Atomic_Num[molecule][atom]-1](aev.float())
            molecule_out = molecule_out + net_out
        
        targett = Target[molecule]
        targett = torch.from_numpy(np.array(targett))
        targett = targett.float()
        targett = targett.view(1, -1)
        loss = criterion(molecule_out, targett)
        if ((targett-15) <= molecule_out <= (targett+15)) :
            answ = answ + 1
    
    print(answ)
doNN()

epoch:  0
epoch:  1
epoch:  2
epoch:  3
epoch:  4
epoch:  5
epoch:  6
epoch:  7
epoch:  8
epoch:  9
epoch:  10
epoch:  11
epoch:  12
epoch:  13
epoch:  14
epoch:  15
epoch:  16
epoch:  17
epoch:  18
epoch:  19
1233


# Choice of Loss function , optimization and Results

Loss Funtions that worked (for us) : L1Loss(), SmoothL1Loss(), HingeEmbeddingLoss() and similar loss functions
    MSELoss() blows up loss to nan
    loss with log functions give errors

Optimization - Adam (as suggested in the ANI1 -paper);SGD - gives poor results
lr(learning rate) = 0.0099
beta1 = 0.5 to 0.9 (0.5 gave slightly better results)
beta2 = 0.59 to 0.99 (0.59 gave slightly better results)


# results

Best result :
    out of 2100 test molecules, predicted atomization energies : -
    
    2094 are within absolute difference 90 range
    1954 are within absolute difference 40 range
    1233 are within absolute difference 15 range

parameter for best result : for optimizer lr = 0.0099 , betas=(0.5,0.59), eps = 1e-8.    
    
    